In [ ]:
import cv2
import numpy as np

# Taking input
image1 = cv2.imread('S2.jpeg')
image2 = cv2.imread('S1.jpeg')

#  convert to grayscale
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Initialize feature detector
detector = cv2.SIFT_create()


# Find keypoints and descriptors
kp1, des1 = detector.detectAndCompute(gray1, None)
kp2, des2 = detector.detectAndCompute(gray2, None)

# Create a feature matcher
bf = cv2.BFMatcher()

# Match features between images
matches = bf.knnMatch(des1, des2, k=2)

# Apply ratio test to filter matches
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Extract matched keypoints
src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Find the homography matrix
#H, _ = cv2.findHomography(src_pts, dst_pts)
H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# Warp image1 to image2's perspective
result = cv2.warpPerspective(image1, H, (image1.shape[1] + image2.shape[1], image1.shape[0]))

# Combine both images
result[:, 0:image2.shape[1]] = image2

# Save the stitched image
cv2.imwrite('panoramaS1_S2.jpeg', result)


True